In [1]:
import urllib.request
import sys
import validators
from bs4 import BeautifulSoup
import pandas as pd
from urllib.request import urlopen
import requests
import re
import itertools
from pandas import DataFrame
import pandas as pd
import csv 
from urllib.parse import urlparse

In [2]:
#dict of all high schools 

#pleasanton
bay_area = {'amador-valley-': 'dons-(pleasanton,ca)/', 'foothill-': 'falcons-(pleasanton,ca)/'}
#dublin
bay_area['dublin-'] = 'gaels-(dublin,ca)/'
#Danville
bay_area['monta-vista-'] = 'matadors-(cupertino,ca)/'
#san ramon
bay_area['san-ramon-valley-'] = 'the-wolves-(danville,ca)/'
bay_area['dougherty-valley-'] = 'wildcats-(san-ramon,ca)/'
bay_area['california-'] = 'grizzlies-(san-ramon,ca)/'
#mountain house
bay_area['mountain-house-'] = 'mustangs-(mountain-house,ca)/'
#livermore
bay_area['livermore-'] = 'cowboys-(livermore,ca)/'
bay_area['granada-'] = 'matadors-(livermore,ca)/'
#athenian
#Carondelet
#de la salle

bay_area

{'amador-valley-': 'dons-(pleasanton,ca)/',
 'foothill-': 'falcons-(pleasanton,ca)/',
 'dublin-': 'gaels-(dublin,ca)/',
 'monta-vista-': 'matadors-(cupertino,ca)/',
 'san-ramon-valley-': 'the-wolves-(danville,ca)/',
 'dougherty-valley-': 'wildcats-(san-ramon,ca)/',
 'california-': 'grizzlies-(san-ramon,ca)/',
 'mountain-house-': 'mustangs-(mountain-house,ca)/',
 'livermore-': 'cowboys-(livermore,ca)/',
 'granada-': 'matadors-(livermore,ca)/'}

In [3]:
#list of all sports 
fall_sports = ["water-polo-" , "cross-country-"]
spring_sports = [ "lacrosse-", "baseball-", "softball-", "tennis-", "track-", "volleyball-"]
winter_sports = ["soccer-", "basketball-", "wrestling-"]

In [4]:
#years 
winter_sports_timing = ["winter-18-19", "winter-18-20"]
spring_sports_timing = ["spring-19", "spring-20"]
fall_sports_timing = ["fall-18", "fall-19", "fall-20"]

In [5]:
#JV and Varsity 
levels = ["jv-", "freshman-", ""]

#girls and boys 
genders = ["boys-", "girls-"]

In [6]:
def find_names(html, school, year_of_playing):
    
    #initialize data frame to store all the names 
    names_years_school = pd.DataFrame()
    #pattern on the website - THIS VALUE NEEDS TO BE CHANGED
    #ONLY If it doesnt run
    pattern = "StyledAnchor-jb44mu-0 ewnKAc"
    
    #initialize name and schools list to hold information
    names = []
    schools = []
    calander_years = []
    
    #traverse and find all the student names
    for match in re.finditer(pattern, html):
        starting11 = match.end() + 2
        ending11 = match.end() + 30
        name = html[starting11 : ending11]
        name = name.split("</")[0]
        name = name.replace("O&#x27;", "O'")
        name_holder = name.split(' ')
        if len(name_holder) > 2:
            continue
        name = name_holder[1]
        #creating a list of schools (to store with df)
        calander_years.append(year_of_playing)
        schools.append(school)
        names.append(name)
            
    #intializing list to hold class year 
    list_of_years = []
    pattern1 = "classYear"
    #to keep track of index 
    count = 0
    for match in re.finditer(pattern1, html):
        if count >= len(names):
                break
        class_year = match.end() + 2
        year = html[class_year:class_year+2]
        #find length of last name to move the index 
        length_of_lastname = len(names[count])
        end_last_name = match.start() - 3
        start_of_lastname = match.start()-length_of_lastname-3
        #Save last name to compare against previous list created
        last_name = html[start_of_lastname:end_last_name]
        if names[count] == last_name:
            #add years to list 
            list_of_years.append(year)
            count += 1
    #Create panda 
    names_years_school = pd.DataFrame(list(zip(names, list_of_years, schools, calander_years)), columns = ['LastName', 'Year', 'School', 'Year of Playing'])    
             

    return names_years_school

In [7]:
#create html page for each link 
def create_html(link):
    url = link
    page = urlopen(url)
    html_bytes = page.read()
    html = html_bytes.decode("utf-8")
    return html


In [8]:
#FALL SPORTS GENERATOR 
fall_links = {}
for school in bay_area:
    for sport in fall_sports:
        for time in fall_sports_timing:
            for level in levels:
                for gender in genders:
                    #create links with different sports/schools/years/gender 
                    link = "https://www.maxpreps.com/high-schools/" + school + bay_area[school] + gender + level + sport + time + "/roster.htm"
                    request = requests.get(link)
                    if request.status_code == 200:
                        if school in fall_links:
                            fall_links[school].append(link)
                        else:
                            fall_links[school] = [link]
fall_links           

{'amador-valley-': ['https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/girls-jv-water-polo-fall-18/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/boys-water-polo-fall-18/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/girls-water-polo-fall-18/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/girls-jv-water-polo-fall-19/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/boys-water-polo-fall-19/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/girls-water-polo-fall-19/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/boys-jv-water-polo-fall-20/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/girls-jv-water-polo-fall-20/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(

In [9]:
#dataframe to store all the names
student_names_df = pd.DataFrame()

In [10]:
for school in fall_links:
    for link in fall_links[school]:
        #store list of names into list
        calander_year = link[-13:-11]
        year_of_playing = "fall " + calander_year
        
        df1 = find_names(create_html(link), school, year_of_playing)
        print(df1)
        if df1.empty:
            continue
        mix = [df1, student_names_df]
        student_names_df = pd.concat(mix)
student_names_df

        LastName Year          School Year of Playing
0        Shiblaq   11  amador-valley-         fall 18
1          Klein   11  amador-valley-         fall 18
2       Diligent   11  amador-valley-         fall 18
3         Walder   9,  amador-valley-         fall 18
4      Schurhoff   11  amador-valley-         fall 18
5       Bouchard   11  amador-valley-         fall 18
6   Ellinghuysen   10  amador-valley-         fall 18
7           Luck   9,  amador-valley-         fall 18
8        Berglin   9,  amador-valley-         fall 18
9        Jessica   10  amador-valley-         fall 18
10          Gill   10  amador-valley-         fall 18
11       Medinas   9,  amador-valley-         fall 18
12        Keegan   10  amador-valley-         fall 18
13         Grell   11  amador-valley-         fall 18
14      Lovelock   10  amador-valley-         fall 18
15     McClellan   9,  amador-valley-         fall 18
16      Krishnan   9,  amador-valley-         fall 18
17         Quinn   9,  amado

Empty DataFrame
Columns: [LastName, Year, School, Year of Playing]
Index: []
   LastName Year     School Year of Playing
0    Ambriz   10  foothill-         fall 19
1    Kruger   11  foothill-         fall 19
2   Goodman   12  foothill-         fall 19
3   Minnite   12  foothill-         fall 19
4     Menon   12  foothill-         fall 19
5      Tran   12  foothill-         fall 19
6        Xu   12  foothill-         fall 19
7      Zhao   12  foothill-         fall 19
8   Kennedy   10  foothill-         fall 19
9     Jones   11  foothill-         fall 19
10      Kim   10  foothill-         fall 19
11  Koempel   12  foothill-         fall 19
12     Chen   12  foothill-         fall 19
13   Sharma   12  foothill-         fall 19
    LastName Year     School Year of Playing
0     Dillon   12  foothill-         fall 19
1     Fluker   12  foothill-         fall 19
2       Reid   12  foothill-         fall 19
3       Ynez   12  foothill-         fall 19
4      Chung   12  foothill-         f

Empty DataFrame
Columns: [LastName, Year, School, Year of Playing]
Index: []
     LastName Year             School Year of Playing
0      Mckeon   10  san-ramon-valley-         fall 19
1   Steingraf   11  san-ramon-valley-         fall 19
2        Webb   12  san-ramon-valley-         fall 19
3    Alamillo   9,  san-ramon-valley-         fall 19
4      Watson   11  san-ramon-valley-         fall 19
5    Protenic   11  san-ramon-valley-         fall 19
6   Mundelius   11  san-ramon-valley-         fall 19
7       Riley   12  san-ramon-valley-         fall 19
8      Schade   11  san-ramon-valley-         fall 19
9       Libby   11  san-ramon-valley-         fall 19
10      Haket   nu  san-ramon-valley-         fall 19
11      Neach   12  san-ramon-valley-         fall 19
12  Decredico   12  san-ramon-valley-         fall 19
13     Nuelle   nu  san-ramon-valley-         fall 19
14   Flanders   11  san-ramon-valley-         fall 19
15     Bugbee   10  san-ramon-valley-         fall 19
16   

Empty DataFrame
Columns: [LastName, Year, School, Year of Playing]
Index: []
Empty DataFrame
Columns: [LastName, Year, School, Year of Playing]
Index: []
Empty DataFrame
Columns: [LastName, Year, School, Year of Playing]
Index: []
Empty DataFrame
Columns: [LastName, Year, School, Year of Playing]
Index: []
Empty DataFrame
Columns: [LastName, Year, School, Year of Playing]
Index: []
Empty DataFrame
Columns: [LastName, Year, School, Year of Playing]
Index: []
Empty DataFrame
Columns: [LastName, Year, School, Year of Playing]
Index: []
Empty DataFrame
Columns: [LastName, Year, School, Year of Playing]
Index: []
Empty DataFrame
Columns: [LastName, Year, School, Year of Playing]
Index: []
Empty DataFrame
Columns: [LastName, Year, School, Year of Playing]
Index: []
Empty DataFrame
Columns: [LastName, Year, School, Year of Playing]
Index: []
Empty DataFrame
Columns: [LastName, Year, School, Year of Playing]
Index: []
      LastName Year       School Year of Playing
0       Miller   12  califo

    LastName Year      School Year of Playing
0    Sommers   12  livermore-         fall 18
1     Pettit   9,  livermore-         fall 18
2     Wapman   11  livermore-         fall 18
3     McNeil   11  livermore-         fall 18
4     Guidry   9,  livermore-         fall 18
5     Fuller   11  livermore-         fall 18
6    Hawkins   12  livermore-         fall 18
7       Lein   12  livermore-         fall 18
8     Guidry   11  livermore-         fall 18
9   Hamilton   10  livermore-         fall 18
10     Silva   10  livermore-         fall 18
11  Hornbeck   10  livermore-         fall 18
12     Suazo   12  livermore-         fall 18
13   Zuluaga   12  livermore-         fall 18
Empty DataFrame
Columns: [LastName, Year, School, Year of Playing]
Index: []
Empty DataFrame
Columns: [LastName, Year, School, Year of Playing]
Index: []
Empty DataFrame
Columns: [LastName, Year, School, Year of Playing]
Index: []
Empty DataFrame
Columns: [LastName, Year, School, Year of Playing]
Index: []
Em

,LastName,Year,School,Year of Playing
0,Wolf,11,granada-,fall 20
1,Stewart,11,granada-,fall 20
0,Wolf,10,granada-,fall 19
1,Memula,12,granada-,fall 19
2,Katen,12,granada-,fall 19
...,...,...,...,...
15,McClellan,"9,",amador-valley-,fall 18
16,Krishnan,"9,",amador-valley-,fall 18
17,Quinn,"9,",amador-valley-,fall 18
18,Baer,"9,",amador-valley-,fall 18


In [11]:
#SPRING SPORTS LINKS
spring_links = {}
for school in bay_area:
    for sport in spring_sports:
        for time in spring_sports_timing:
            for level in levels:
                for gender in genders:
                    link = "https://www.maxpreps.com/high-schools/" + school + bay_area[school] + gender + level + sport + time + "/roster.htm"
                    #print(link)
                    request = requests.get(link)
                    if request.status_code == 200:
                        if school in spring_links:
                            spring_links[school].append(link)
                        else:
                            spring_links[school] = [link]
spring_links           

{'amador-valley-': ['https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/boys-jv-lacrosse-spring-19/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/girls-jv-lacrosse-spring-19/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/boys-lacrosse-spring-19/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/girls-lacrosse-spring-19/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/boys-jv-lacrosse-spring-20/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/girls-jv-lacrosse-spring-20/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/boys-lacrosse-spring-20/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/girls-lacrosse-spring-20/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(p

In [12]:
#GENERATE TABLE OF SPRING SPORTS

for school in spring_links:
    for link in spring_links[school]:
        #store list of names into list
        calander_year = link[-13:-11]
        year_of_playing = "spring " + calander_year
                
        df1 = find_names(create_html(link), school, year_of_playing)
        if df1.empty:
            continue
        mix = [df1, student_names_df]
        student_names_df = pd.concat(mix)
student_names_df

,LastName,Year,School,Year of Playing
0,Cardenas,11,granada-,spring 20
1,Moen,12,granada-,spring 20
2,Nguyen,12,granada-,spring 20
3,Ghafoori,12,granada-,spring 20
4,Johansson,11,granada-,spring 20
...,...,...,...,...
15,McClellan,"9,",amador-valley-,fall 18
16,Krishnan,"9,",amador-valley-,fall 18
17,Quinn,"9,",amador-valley-,fall 18
18,Baer,"9,",amador-valley-,fall 18


In [13]:
#WINTER SPORTS TABLE 
winter_links = {}
for school in bay_area:
    for sport in winter_sports:
        for time in winter_sports_timing:
            for level in levels:
                for gender in genders:
                    link = "https://www.maxpreps.com/high-schools/" + school + bay_area[school] + gender + level + sport + time + "/roster.htm"
                    request = requests.get(link)
                    if request.status_code == 200:
                        if school in winter_links:
                            winter_links[school].append(link)
                        else:
                            winter_links[school] = [link]
winter_links 

{'amador-valley-': ['https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/boys-jv-soccer-winter-18-19/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/girls-jv-soccer-winter-18-19/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/boys-freshman-soccer-winter-18-19/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/girls-freshman-soccer-winter-18-19/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/boys-soccer-winter-18-19/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/girls-soccer-winter-18-19/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/boys-jv-basketball-winter-18-19/roster.htm',
  'https://www.maxpreps.com/high-schools/amador-valley-dons-(pleasanton,ca)/girls-jv-basketball-winter-18-19/roster.htm',
  'https://www.maxpreps.com/

In [14]:
#GENERATE TABLE FOR WINTER SPORTS 

for school in winter_links:
    for link in winter_links[school]:
        #store list of names into list
        calander_year = link[-13:-11]
        year_of_playing = "winter " + calander_year
        
        
        df1 = find_names(create_html(link), school, year_of_playing)
        if df1.empty:
            continue
        mix = [df1, student_names_df]
        student_names_df = pd.concat(mix)

In [15]:
student_names_df

,LastName,Year,School,Year of Playing
0,Lisondra,11,granada-,winter 19
1,Molitoris,"9,",granada-,winter 19
2,Price,11,granada-,winter 19
3,Faiaipau,11,granada-,winter 19
4,Holt,11,granada-,winter 19
...,...,...,...,...
15,McClellan,"9,",amador-valley-,fall 18
16,Krishnan,"9,",amador-valley-,fall 18
17,Quinn,"9,",amador-valley-,fall 18
18,Baer,"9,",amador-valley-,fall 18


In [16]:
#CONVERTING TO A CSV 
student_names_df.to_csv(r'/Users/aishvishah/Documents/brightfox/max_prep_names.csv')